In [1]:
# lassen wir mal 
# https://towardsdatascience.com/time-series-classification-for-human-activity-recognition-with-lstms-using-tensorflow-2-and-keras-b816431afdff 
# probieren

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder
import random
import scipy.stats as stats
import tensorflow.keras.backend as K
from tensorflow import keras
import tensorflow as tf

In [3]:
df = pd.read_csv('../pdiot-practical/pdiot-data/2021/Respeck_recordings_clean.csv')
df.shape

/home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(492676, 13)

In [4]:
class_labels = {
    'Climbing stairs': 0,
 'Descending stairs': 1,
 'Desk work': 2,
 'Falling on knees': 3,
 'Falling on the back': 4,
 'Falling on the left': 5,
 'Falling on the right': 6,
 'Lying down left': 7,
 'Lying down on back': 8,
 'Lying down on stomach': 9,
 'Lying down right': 10,
 'Movement': 11,
 'Running': 12,
 'Sitting': 13,
 'Sitting bent backward': 14,
 'Sitting bent forward': 15,
 'Standing': 16,
 'Walking at normal speed': 17
}
df['activity_type'] = df['activity_type'].replace(class_labels)

In [5]:
len(df['subject_id'].unique())

46

In [6]:
train_subject_list = random.sample(df['subject_id'].unique().tolist(), 37)
df_train = df[df['subject_id'].isin(train_subject_list)]
df_test = df[~df['subject_id'].isin(train_subject_list)]

In [7]:
scale_columns = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
scaler = RobustScaler()
scaler = scaler.fit(df_train[scale_columns])

df_train.loc[:, scale_columns] = scaler.transform(
  df_train[scale_columns].to_numpy()
)

df_test.loc[:, scale_columns] = scaler.transform(
  df_test[scale_columns].to_numpy()
)

/home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  "X does not have valid feature names, but"
/home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  "X does not have valid feature names, but"
/home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCo

In [16]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return tf.convert_to_tensor(Xs), tf.reshape(tf.convert_to_tensor((ys)), (-1, 1))

In [17]:
TIME_STEPS = 40
STEP = 20

X_train, y_train = create_dataset(
    df_train[scale_columns],
    df_train.activity_type,
    TIME_STEPS,
    STEP
)

X_test, y_test = create_dataset(
    df_test[scale_columns],
    df_test.activity_type,
    TIME_STEPS,
    STEP
)

In [18]:
print(X_train.shape, y_train.shape)

(19934, 40, 6) (19934, 1)


In [19]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

In [20]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

In [21]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

Epoch 1/20


2021-11-08 18:07:43.289771: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-08 18:07:43.290466: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1190000000 Hz


NotImplementedError: in user code:

    /home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)